# Part 3: Advanced Analysis

In this part, we will implement advanced analysis techniques for physiological time series data, including time-domain feature extraction, frequency analysis, and wavelet transforms.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
import pywt
import os

# Set plotting style
plt.style.use("seaborn-v0_8")
%matplotlib inline
sns.set_context('notebook')

## 1. Time-Domain Feature Extraction

Implement the `extract_time_domain_features` function to extract various time-domain features from physiological signals.

In [3]:
import pandas as pd
import numpy as np

def extract_time_domain_features(data, window_size=60):
    """Extract time-domain features from physiological signals.
    
    Parameters
    -----------
    data : pandas.DataFrame
        Input data with columns: ['timestamp', 'heart_rate', 'eda', 'temperature', 'subject_id', 'session']
    window_size : int, optional
        Size of the rolling window in seconds, default=60
        
    Returns
    --------
    pandas.DataFrame
        DataFrame containing extracted features for each window
    """
    # Convert window_size from seconds to number of samples
    # Assuming data is sampled at 1 Hz (1 sample per second)
    window_samples = window_size
    
    # Initialize DataFrame for features
    features = pd.DataFrame()

    # Basic statistics using rolling window
    features['mean'] = data['heart_rate'].rolling(window=window_samples).mean()
    features['std'] = data['heart_rate'].rolling(window=window_samples).std()
    features['min'] = data['heart_rate'].rolling(window=window_samples).min()
    features['max'] = data['heart_rate'].rolling(window=window_samples).max()

    # Heart rate statistics
    features['mean_hr'] = data['heart_rate'].rolling(window=window_samples).mean()
    features['std_hr'] = data['heart_rate'].rolling(window=window_samples).std()

    # Beat-to-beat variability (RR intervals)
    rr_intervals = 60000 / data['heart_rate']  # Convert HR to RR intervals in ms

    # Calculate successive differences within each window
    rr_diff = rr_intervals.diff()

    # RMSSD (Root Mean Square of Successive Differences)
    features['rmssd'] = np.sqrt(rr_diff.rolling(window=window_samples).apply(lambda x: np.mean(x**2)))

    # SDNN (Standard Deviation of NN intervals)
    features['sdnn'] = rr_intervals.rolling(window=window_samples).std()

    # pNN50 (Percentage of successive RR intervals differing by >50ms)
    features['pnn50'] = rr_diff.rolling(window=window_samples).apply(
        lambda x: 100 * np.sum(np.abs(x) > 50) / len(x) if len(x) > 0 else 0
    )

    # Drop NaN values from rolling window calculations
    features = features.dropna()
    
    # Adding units to feature names
    features = features.rename(columns={
        'mean': 'mean_hr_bpm',
        'std': 'std_hr_bpm',
        'min': 'min_hr_bpm',
        'max': 'max_hr_bpm',
        'mean_hr': 'mean_hr_bpm',
        'std_hr': 'std_hr_bpm',
        'rmssd': 'rmssd_ms',
        'sdnn': 'sdnn_ms',
        'pnn50': 'pnn50_percent'
    })
    
    return features


In [5]:
def load_processed_data(processed_dir='data/processed'):
    """Load and concatenate all processed data CSVs."""
    all_files = [f for f in os.listdir(processed_dir) if f.endswith('.csv')]
    dfs = []
    for file in all_files:
        df = pd.read_csv(os.path.join(processed_dir, file))
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

df = load_processed_data()

heart_rate_data = df[['timestamp', 'heart_rate']].dropna()  # Remove NaN values

# Extract time-domain features with a window size of 60 seconds
time_domain_features = extract_time_domain_features(heart_rate_data, window_size=60)

# Show the resulting features DataFrame
print(time_domain_features.head())


    mean_hr_bpm  std_hr_bpm  min_hr_bpm  max_hr_bpm  mean_hr_bpm  std_hr_bpm  \
69   103.575320    3.166833      92.144   109.03125   103.575320    3.166833   
70   103.436761    3.105347      92.144   107.93950   103.436761    3.105347   
71   103.471394    3.061761      92.144   107.93950   103.471394    3.061761   
72   103.453879    3.075626      92.144   107.93950   103.453879    3.075626   
73   103.530704    2.940071      92.144   107.93950   103.530704    2.940071   

     rmssd_ms    sdnn_ms  pnn50_percent  
69  12.410593  18.517500       1.666667  
70  11.686480  18.215249       1.666667  
71   8.948408  17.959284       0.000000  
72   8.643004  18.035993       0.000000  
73   7.395482  17.191729       0.000000  


## 2. Frequency Analysis

Implement the `analyze_frequency_components` function to perform frequency-domain analysis on the signals.

In [9]:
def analyze_frequency_components(data, sampling_rate, window_size=60, output_dir='results'):
    """Perform frequency-domain analysis on physiological signals.
    
    Parameters:
    -----------
    data : pandas.DataFrame
        Input data with columns: ['timestamp', 'heart_rate', 'eda', 'temperature', 'subject_id', 'session']
    sampling_rate : float
        Sampling rate of the signal in Hz
    window_size : int, optional
        Size of the analysis window in seconds, default=60
    output_dir : str
        Directory to save analysis results
        
    Returns:
    --------
    dict
        Dictionary containing frequency analysis results
    """
    # Convert window_size from seconds to number of samples
    window_samples = int(window_size * sampling_rate)
    
    # Initialize results dictionary
    results = {}
    
    # Process data in windows
    n_windows = len(data) // window_samples
    all_frequencies = []
    all_power = []
    
    # Loop over windows and compute the PSD for each window
    for i in range(n_windows):
        window_data = data['heart_rate'].iloc[i*window_samples:(i+1)*window_samples]
        
        # Check if the window is valid (not empty)
        if len(window_data) < window_samples:
            print(f"Warning: Window {i} is too small, skipping...")
            continue
        
        # Calculate PSD using Welch's method
        frequencies, power = signal.welch(
            window_data,
            fs=sampling_rate,
            nperseg=window_samples
        )
        
        # Handle NaN values in the power spectrum
        power = np.nan_to_num(power)
        
        all_frequencies.append(frequencies)
        all_power.append(power)
    
    # Average results across windows
    results['frequencies'] = np.mean(all_frequencies, axis=0)
    results['power'] = np.mean(all_power, axis=0)
    
    # Define frequency bands
    bands = {
        'VLF': (0.003, 0.04),
        'LF': (0.04, 0.15),
        'HF': (0.15, 0.4)
    }
    
    # Calculate power in each band
    results['bands'] = {}
    for band_name, (low, high) in bands.items():
        mask = (results['frequencies'] >= low) & (results['frequencies'] <= high)
        results['bands'][band_name] = np.sum(results['power'][mask])
    
    # Calculate LF/HF ratio
    results['bands']['LF/HF'] = results['bands']['LF'] / results['bands']['HF']
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Save the frequency analysis results to a file
    file_path = os.path.join(output_dir, 'frequency_analysis_results.npz')
    np.savez(file_path, frequencies=results['frequencies'], power=results['power'], bands=results['bands'])
    
    # Return the results dictionary
    return results

In [10]:
sampling_rate = 1
results = analyze_frequency_components(df, sampling_rate=sampling_rate, window_size=60)
print("Frequencies:", results['frequencies'])
print("Power Spectrum:", results['power'])
print("Power in Bands:", results['bands'])

Frequencies: [0.         0.01666667 0.03333333 0.05       0.06666667 0.08333333
 0.1        0.11666667 0.13333333 0.15       0.16666667 0.18333333
 0.2        0.21666667 0.23333333 0.25       0.26666667 0.28333333
 0.3        0.31666667 0.33333333 0.35       0.36666667 0.38333333
 0.4        0.41666667 0.43333333 0.45       0.46666667 0.48333333
 0.5       ]
Power Spectrum: [2.42500611e+01 2.02959384e+02 3.50286751e+01 6.16905849e+00
 2.40114986e+00 9.15700935e-01 3.94089041e-01 3.20582525e-01
 2.33316806e-01 1.54621839e-01 1.10042100e-01 8.45787088e-02
 7.86833241e-02 7.58325358e-02 6.00559930e-02 2.71567503e-02
 3.03198200e-02 3.65697700e-02 3.14062517e-02 4.10020397e-02
 5.20363629e-02 4.90619325e-02 4.62397153e-02 6.98359031e-02
 7.00608492e-02 8.31668024e-02 6.38772233e-02 3.95099362e-02
 6.54156915e-02 5.70124876e-02 5.77926412e-02]
Power in Bands: {'VLF': 237.9880595252225, 'LF': 10.433897654273556, 'HF': 1.0175038953888316, 'LF/HF': 10.2544056111808}


## 3. Time-Frequency Analysis

Implement the `analyze_time_frequency_features` function to analyze time-frequency features using wavelet transforms.

In [11]:
def analyze_time_frequency_features(data, sampling_rate, window_size=60, output_dir='results'):
    """Analyze time-frequency features using wavelet transforms.
    
    Parameters:
    -----------
    data : pandas.DataFrame
        Input data with columns: ['timestamp', 'heart_rate', 'eda', 'temperature', 'subject_id', 'session']
    sampling_rate : float
        Sampling rate of the signal in Hz
    window_size : int, optional
        Size of the analysis window in seconds, default=60
    output_dir : str, optional
        Directory to save analysis results, default='results'
        
    Returns:
    --------
    dict
        Dictionary containing time-frequency analysis results
    """
    # Convert window_size from seconds to number of samples
    window_samples = int(window_size * sampling_rate)
    
    # Initialize results dictionary
    results = {}
    
    # Define wavelet scales (frequency bands)
    scales = np.arange(1, 128)
    results['scales'] = scales
    
    # Process data in windows
    n_windows = len(data) // window_samples
    all_coefficients = []
    all_energy = []
    
    for i in range(n_windows):
        window_data = data['heart_rate'].iloc[i*window_samples:(i+1)*window_samples]
        
        # Check if window contains sufficient data for wavelet transform
        if len(window_data) < window_samples:
            print(f"Warning: Window {i} is too small, skipping...")
            continue
        
        # Apply continuous wavelet transform (CWT) using Morlet wavelet
        coefficients, frequencies = pywt.cwt(
            window_data,
            scales,
            'morl',
            sampling_period=1/sampling_rate
        )
        
        all_coefficients.append(coefficients)
        all_energy.append(np.abs(coefficients)**2)  # Energy is the square of the coefficient magnitude
    
    # Average results across windows
    results['coefficients'] = np.mean(all_coefficients, axis=0)
    results['time_frequency_energy'] = np.mean(all_energy, axis=0)
    
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    # Save the results as an .npz file
    file_path = os.path.join(output_dir, 'wavelet_analysis_results.npz')
    np.savez(file_path, scales=results['scales'], coefficients=results['coefficients'], energy=results['time_frequency_energy'])
    
    # Return the results dictionary
    return results

In [12]:
results = analyze_time_frequency_features(df, sampling_rate, window_size=60)

## Example Usage

Here's how to use these functions with your data:

In [ ]:
# Load your data
data = pd.read_csv('data/processed/S1_processed.csv')

# Extract time-domain features
features = extract_time_domain_features(data, window_size=60)
print("Time-domain features:")
print(features.head())

# Analyze frequency components
sampling_rate = 4.0  # Hz
freq_results = analyze_frequency_components(data, sampling_rate, window_size=60)
print("\nFrequency analysis results:")
print("Frequency bands:", freq_results['bands'])

# Analyze time-frequency features
tf_results = analyze_time_frequency_features(data, sampling_rate, window_size=60)
print("\nTime-frequency analysis results:")
print("Wavelet scales:", tf_results['scales'].shape)
print("Coefficients shape:", tf_results['coefficients'].shape)